In [100]:
from bs4 import BeautifulSoup
import requests
import re
import openai
import csv
import ast
import openpyxl

In [93]:
def scrape(url):
    if url:
        try:
            r=requests.get(url)
            if r.status_code==200:
                soup=BeautifulSoup(r.text,"lxml")
                text=[]
                #<p>tags
                p_tags=[]
                max_tags = 20

                names=soup.find_all("p")
                for i, tag in enumerate(names):
                    if i >= max_tags:
                        break  # Stop after extracting the first 7 tags
                    if tag.text!='':
                        name=tag.text
                        p_tags.append(name)
                text+=[str(item) for item in p_tags]
                
                #<a>tags
                a_tags=[]
                names=soup.find_all("a")
                for i, tag in enumerate(names):
                    if i >= max_tags:
                        break  # Stop after extracting the first 7 tags
                    if tag.text!='':
                        name=tag.text
                        a_tags.append(name)
                
                text += [str(item) for item in a_tags]
                #hrefs
                hrefs = [a.get('href') for a in names]
                social_media_keywords = [
                "facebook.com",
                "linkedin.com",
                "instagram.com",
                "pinterest.com",
                "snapchat.com",
                "tiktok.com",
                "reddit.com",
                ]
                    # Initialize a list to store the selected social media links
                selected_social_media_links = []

                # Iterate through the hrefs and filter social media links
                for href in hrefs:
                    href=str(href)
                    # Check if any of the social media keywords is present in the URL
                    if any(keyword in href.lower() for keyword in social_media_keywords):
                        selected_social_media_links.append(href)

                # Now, 'selected_social_media_links' contains only the complete social media links

                # Convert the selected social media links to a list of strings
                text = [str(item) for item in selected_social_media_links]
                #<h2>tags
                h2_tags=[]
                names=soup.find_all("h2")
                for i, tag in enumerate(names):
                    if i >= max_tags:
                        break  # Stop after extracting the first 7 tags
                    if tag.text!='':
                        name=tag.text
                        h2_tags.append(name)
                text+=[str(item) for item in h2_tags]
                return text
            else:
                print(f"Skipping {url} due to non-200 status code: {response.status_code}")
        except Exception as e:
            print(f"An error occurred while connecting to {url}: {str(e)}")
    

In [96]:
def to_csv(input_string):
    data_dict = ast.literal_eval(input_string)

    # Define the desired CSV file path
    csv_file_path = 'output.csv'

    # Extract keys and values from the dictionary
    keys = list(data_dict.keys())
    values = list(data_dict.values())

    # Check if the CSV file exists
    csv_file_exists = False
    try:
        with open(csv_file_path, 'r') as file:
            csv_reader = csv.reader(file)
            if any(row for row in csv_reader):
                csv_file_exists = True
    except FileNotFoundError:
        pass

    # Write the data to a CSV file
    with open(csv_file_path, 'a', newline='') as file:
        csv_writer = csv.writer(file)
        
        # Write header row only if the file is newly created
        if not csv_file_exists:
            csv_writer.writerow(keys)
        
        # Write data row with values
        csv_writer.writerow(values)

    print(f'Data has been appended to CSV file "{csv_file_path}".')

In [81]:
def ai(file_path):
    try:
        openai.api_key = '        # Call the OpenAI API to generate scenario information from the prompt
        # Read the contents of the file into a string
        with open(file_path, 'r') as file:
            file_contents = file.read()
        response = openai.Completion.create(
                engine="text-davinci-003",
                prompt=file_contents,
                max_tokens=500,
                stop=None,
                temperature=0.7,
            )
            # Extract relevant information from the API response\
        extracted_response=response.choices[0].text
        print(extracted_response)
        return extracted_response
    except Exception as e:
        print(f"An error occurred while calling the OpenAI API: {str(e)}")
        return None

In [111]:
file_path = "URLs.txt"
# Read the contents of the file into an array (list)
array_contents=[]
with open(file_path, 'r') as file:
 for line in file:
    # Append each line to the list
    array_contents.append(line.strip())
tool_info = {
    "url": "write url of the website here",
    "short-description": "write a short description about the tool here",
    "key-features": "Identify and extract the main features and functionalities the tool offers and write here",
    "Use cases": "Recognize and list various scenarios or domains where the tool can be applied effectively here",
    "Price model": "if no information available then write (free) here",
    "Social Links": "Extract links to the tool's social media profiles (if present) and write here",
    "Categories": "Identify and extract the categories or domains the tool falls under"
}

file_path="nwes.txt"
for i in array_contents:
    arr= ["Read the following data taken from various tags from the html of webiste: " ]
    arr.append(i)
    arr.append(scrape(i))
    arr.append("--- give me an answer in the following format that i can add into a csv file with no more than 1000 characters: ")
    arr.append(tool_info)
    list_as_strings = [str(item) for item in arr]
            # Write the list elements to the text file
    with open(file_path, 'w') as file:
            pass
            for item in list_as_strings:
                item_without_brackets_and_newlines = item.replace('[', '').replace(']', '').replace('\n', '')
                # Write each element to the file without newline characters
                file.write(item_without_brackets_and_newlines + '\n')

    to_csv(ai(file_path))


{'url': 'https://www.synthesia.io/', 'short-description': 'Synthesia is a video creation tool for teams to create training videos at scale', 'key-features': 'Replace boring text, PowerPoints and PDFs with engaging videos, create high-quality sales enablement videos, transform help articles into short videos, create information security training videos, create professional-looking product explainers, create videos as easily as a slide deck, turn text into high-quality voiceovers with one click, create videos as diverse as your audience, keep video library up to date without reshoots, embed videos into favorite tools, 200+ free video templates, create AI videos', 'Use cases': 'Training, Sales, Help Articles, Information Security Training, Product Explainer Videos, Video Creation, Voiceover', 'Price model': 'Free', 'Social Links': 'https://www.linkedin.com/company/synthesia-technologies/', 'Categories': 'Video Creation, Voiceover'}
Data has been appended to CSV file "output.csv".
